In [1]:
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT,COMPLEX_MOVEMENT,RIGHT_ONLY
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, VecTransposeImage
from matplotlib import pyplot as plt

c:\Dissertation\dissertation\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Dissertation\dissertation\lib\site-packages\torch\utils\tensorboard\__init__.py:5: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  tensorboard.__version__


In [2]:
# Define a function to create the environment
def make_env():
    env = gym_super_mario_bros.make('SuperMarioBros-v0')
    env = JoypadSpace(env, SIMPLE_MOVEMENT)
    env = GrayScaleObservation(env, keep_dim=True)
    return env

# Create multiple environments using DummyVecEnv
num_envs = 3
env = DummyVecEnv([make_env for _ in range(num_envs)])

# Stack frames for each environment
env = VecFrameStack(env, 4, channels_order='last')
env = VecTransposeImage(env)
state = env.reset()
print(state.shape)
#print(env.observation_space.shape)

c:\Dissertation\dissertation\lib\site-packages\gym\envs\registration.py:594: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  f"The environment {id} is out of date. You should consider "
c:\Dissertation\dissertation\lib\site-packages\gym\core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
c:\Dissertation\dissertation\lib\site-packages\gym\wrappers\step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be

(3, 4, 240, 256)


In [ ]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = GrayScaleObservation(env, keep_dim=True)
env = DummyVecEnv([make_env for _ in range(num_envs)])
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
env = make_env()
state = env.reset()
print(state.shape)

In [ ]:
import time

for _ in range(100):  # Render for 100 steps
    obs = env.render(mode='human')
    actions = [env.action_space.sample() for _ in range(env.num_envs)]
    obs, rewards, dones, info = env.step(actions)

env.close()  # Close the environment after rendering

In [3]:
import os

#Import PPO for algos
from stable_baselines3 import PPO
from stable_baselines3 import DQN

#Import base callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True
    
CHECKPOINT_DIR = './train'
LOG_DIR = './logs'

callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [4]:
model = PPO('CnnPolicy',env,verbose=1,tensorboard_log=LOG_DIR,learning_rate=0.000001,n_steps=128)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [5]:
model.load('./train/best_model_10000')

ValueError: unsupported pickle protocol: 5